# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.nn.functional as F
import torch.optim as optim

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
# Normalize(mean, std)，通过image = (imgae - mean) / std来归一化
# 一般为一个三元组如(0.5,0.5,0.5)，灰度图就一个channel所以一个值
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist/MNIST\raw

Processing...
Done!


C:\Users\aaa\anaconda3\envs\attack_and_defense\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Then, we define the model, object function and optimizer that we use to classify.

In [9]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x) # (n,1,28,28)->(n,32,26,26)
        x = F.relu(x)
        x = self.conv2(x) # (n,32,26,26)->(n,64,24,24)
        x = F.relu(x)
        x = F.max_pool2d(x, 2) # (n,64,24,24)->(n,64,12,12)
        x = torch.flatten(x, 1) # (n,64,12,12)->(n,9216)
        x = self.fc1(x) # (n,9216)->(n,64)
        x = self.fc2(x) # (n,64)->(n,10)
        out = F.softmax(x, dim=1)
        return out
    
model = SimpleNet().to(device)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

Next, we can start to train and evaluate!

In [19]:
# train and evaluate

data_num = 0

train_accuracy = []
test_accuracy = []

for epoch in range(NUM_EPOCHS):
    corrects = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        x, y = images.to(device), labels.to(device)
        data_num += x.shape[0]
        optimizer.zero_grad()
        
        model.train()
        output = model(x)
        
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            model.eval()
            pred = torch.argmax(output, dim=1)
            corrects += (pred==y).float().sum()   # 这个.sum()很重要，要不然train_accuracy是一个(10,8,64)的玩意，我们期待它是(10,)
    train_acc = corrects / float(data_num)
    train_accuracy.append(train_acc.float())
    
    corrects = 0
    data_num = 0
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    for images, labels in tqdm(test_loader):
        x_test, y_test = images.to(device), labels.to(device)
        data_num += x.shape[0]
        model.eval()
        with torch.no_grad():
            output = model(x)
        pred = output.argmax(dim=1)
        corrects += (pred==y).float().sum()
    test_acc = corrects / float(data_num)
    test_accuracy.append(test_acc.float())

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 60.02it/s]


#### Q5:
Please print the training and testing accuracy.

In [44]:
print('Training accuracy:', ['%0.2f%%' % (float(100*x)) for x in train_accuracy])
print('Testing accuracy:',['%0.2f%%' % (float(100*x)) for x in test_accuracy])

Training accuracy: ['98.90%', '84.76%', '84.83%', '84.81%', '84.86%', '84.77%', '84.84%', '84.81%', '84.89%', '84.85%']
Testing accuracy: ['100.00%', '98.44%', '100.00%', '100.00%', '99.22%', '98.44%', '100.00%', '97.66%', '100.00%', '99.22%']
